In [ ]:
#hide
#default_exp read

# nbdev.read
- Reading a notebook, and initial bootstrapping for notebook exporting

In [ ]:
#export
from fastcore.imports import *
from fastcore.foundation import *
from fastcore.utils import *
from fastcore.test import *
from fastcore.script import *

import json,ast

In [ ]:
import time,nbclient,tempfile
from IPython.display import Markdown
from pdb import set_trace

## Reading and executing notebooks

A notebook is just a json file:

In [ ]:
minimal_txt = json.loads(open('../tests/minimal.ipynb').read())

We're create a function that lets use display JSON in a more compact form, so we can take a look at this file:

In [ ]:
def _display_json(js):
    "Formatter to reduce vertical space used by JSON display"
    s = re.sub(r',\s*\n\s*(.*,)', r', \1', json.dumps(js, indent=1))
    s = re.sub(r'\[\s*\n\s*([^,]*)\n\s*\]', r'[ \1 ]', s)
    return Markdown(f"```js\n{s}\n```")

In [ ]:
_display_json(minimal_txt)

```js
{
 "cells": [
  {
   "cell_type": "markdown", "metadata": {},
   "source": [ "# A minimal notebook" ]
  },
  {
   "cell_type": "code", "execution_count": null,
   "metadata": {
    "foo": "bar"
   }, "outputs": [],
   "source": [
    "# Do some arithmetic\n",
    "1+1"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3", "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   }, "file_extension": ".py",
   "mimetype": "text/x-python", "name": "python",
   "nbconvert_exporter": "python", "pygments_lexer": "ipython3",
   "version": "3.8.3"
  }
 }, "nbformat": 4,
 "nbformat_minor": 4
}
```

The important bit for us is the `cells`:

In [ ]:
_display_json(minimal_txt['cells'])

```js
[
 {
  "cell_type": "markdown", "metadata": {},
  "source": [ "# A minimal notebook" ]
 },
 {
  "cell_type": "code", "execution_count": null,
  "metadata": {
   "foo": "bar"
  }, "outputs": [],
  "source": [
   "# Do some arithmetic\n",
   "1+1"
  ]
 }
]
```

The second cell here is a `code` cell, however it contains no outputs, because it hasn't been executed yet. To execute a notebook, we first need to convert it into a format suitable for `nbclient` (which expects some `dict` keys to be available as attrs, and some available as regular `dict` keys). Normally, `nbformat` is used for this step, but it's rather slow and inflexible, so we'll write our own function based on `fastcore`'s handy `dict2obj`, which makes all keys available as both attrs *and* keys.

In [ ]:
#export
class NbCell(AttrDict):
    def __init__(self, idx, cell):
        for k,v in cell.items(): self[k] = v
        self.idx = idx
        if 'source' in self: self.set_source(self.source)

    def __repr__(self): return self.source
    
    def set_source(self, source):
        self.source = ''.join(source)
        if self.cell_type=='code' and self.source[:1]!='%':self.parsed = ast.parse(self.source).body

We use an `AttrDict` subclass which has some basic functionality for accessing notebook cells.

In [ ]:
#export
def dict2nb(js):
    "Convert a dict to an `AttrDict`, "
    nb = dict2obj(js)
    nb.cells = nb.cells.enumerate().starmap(NbCell)
    return nb

We can now convert our JSON into this `nbclient`-compatible format...

In [ ]:
minimal = dict2nb(minimal_txt)

...and execute it:

In [ ]:
nbclient.execute(minimal);

One nice feature of the output of `dict2nb` is that we can still use it as a `dict`, so `display_json` still works as before:

In [ ]:
_display_json(minimal.cells[1].outputs)

```js
[
 {
  "output_type": "execute_result", "metadata": {},
  "data": {
   "text/plain": "2"
  },
  "execution_count": 1
 }
]
```

We can see that the cell has been executed, and the output added back to the `nb`. Since loading JSON and converting to an NB is something we'll do a lot, we'll create a shortcut function for it:

In [ ]:
#export
def read_nb(path):
    "Return notebook at `path`"
    with open(path) as f: return dict2nb(json.loads(f.read()))

In [ ]:
minimal = read_nb('../tests/minimal.ipynb')
f'{minimal.cells[0]}'

'# A minimal notebook'

## Config and init functions

In [ ]:
#export
@call_parse
def nbdev_create_config(
    user:Param("Repo username", str),
    host:Param("Repo hostname", str)='github',
    lib_name:Param("Name of library", str)=None,
    path:Param("Path to create config file", str)='.',
    cfg_name:Param("Name of config file to create", str)='settings.ini',
    branch:Param("Repo branch", str)='master',
    git_url:Param("Repo URL", str)="https://github.com/%(user)s/%(lib_name)s/tree/%(branch)s/",
    custom_sidebar:Param("Create custom sidebar?", bool_arg)=False,
    nbs_path:Param("Name of folder containing notebooks", str)='.',
    lib_path:Param("Folder name of root module", str)='%(lib_name)s',
    doc_path:Param("Folder name containing docs", str)='docs',
    tst_flags:Param("Test flags", str)='',
    version:Param("Version number", str)='0.0.1',
    **kwargs
):
    "Creates a new config file for `lib_name` and `user` and saves it."
    if lib_name is None:
        parent = Path.cwd().parent
        lib_name = parent.parent.name if parent.name=='nbs' else parent.name
    g = locals()
    config = {o:g[o] for o in 'host lib_name user branch git_url lib_path nbs_path doc_path \
        tst_flags version custom_sidebar'.split()}
    config = {**config, **kwargs}
    save_config_file(Path(path)/cfg_name, config)

This is just a wrapper for `fastcore`'s `save_config_file` which sets some `nbdev` defaults.

In [ ]:
nbdev_create_config('fastai', path='..', nbs_path='nbs', tst_flags='tst', cfg_name='test_settings.ini')
cfg = Config(cfg_name='test_settings.ini')
test_eq(cfg.lib_name, 'nbdev')
test_eq(cfg.git_url, "https://github.com/fastai/nbdev/tree/master/")
cwd = Path.cwd()
test_eq(cfg.path('lib_path'), cwd.parent/'nbdev')
test_eq(cfg.path('nbs_path'), cwd)
test_eq(cfg.path('doc_path'), cwd.parent/'docs')

In [ ]:
#export
_init,_version = '__init__.py','version.py'
def update_version(path:Path):
    "Add or update `__version__` in `version.py`"
    path = Path(path)
    (path/_version).write_text(f"__version__='{Config().version}'\n")

In [ ]:
#export
def _has_py(fs): return any(1 for f in fs if f.endswith('.py'))

def add_init(path):
    "Add `__init__.py` in all subdirs of `path` containing python files if it's not there already"
    # we add the lowest-level `__init__.py` files first, which ensures _has_py succeeds for parent modules
    path = Path(path)
    if not (path/_version).exists(): update_version(path)
    if not (path/_init).exists(): (path/_init).write_text("from .version import __version__\n")
    for r,ds,fs in os.walk(path, topdown=False):
        r = Path(r)
        subds = (os.listdir(r/d) for d in ds)
        if _has_py(fs) or any(filter(_has_py, subds)) and not (r/_init).exists(): (r/_init).touch()

In [ ]:
#hide
with tempfile.TemporaryDirectory() as d:
    d = Path(d)
    (d/'a/b').mkdir(parents=True)
    (d/'a/b/f.py').touch()
    (d/'a/c').mkdir()
    add_init(d)
    assert not (d/'a/c'/_init).exists()
    for e in [d, d/'a', d/'a/b']: assert (e/_init).exists(),e

In [ ]:
#export
@with_cast
def cell_header(nb_path:Path, lib_path:Path):
    "Create `#nbdev_cell {source file}` header"
    return f"#nbdev_cell {os.path.relpath(nb_path.resolve(), lib_path)}"

In [ ]:
#export
def export_cells(cells, hdr, file, offset=0):
    "Export `cells` to `file`"
    for cell in cells: file.write(f'{hdr} {cell.idx}\n{cell}\n\n\n')

In [ ]:
#export
def create_all_cell(vs):
    "Create string that defines `__all__` with `vs`"
    return f"#nbdev_cell auto 0\n__all__ = {list(vs)}\n\n\n"

In [ ]:
#export
def risinstance(types, obj=None):
    "Curried `isinstance` but with args reversed, suitable for `partial`"
    if not obj: return partial(risinstance,types)
    return isinstance(obj, types)

In [ ]:
# Fairly basic export to bootstrap nbdev. See next notebook for more complete implementation
def _export_nb(fname, dest):
    # grab the source from all the cells that have an `export` comment
    nb = read_nb(fname)
    cells = L(cell for cell in nb.cells if re.match(r'#\s*export', cell.source))
    
    # find all the exported functions, to create `__all__`:
    funcs = cells.attrgot('parsed').concat().filter(risinstance((ast.FunctionDef,ast.ClassDef))).attrgot('name')
    exp_funcs = [f for f in funcs if f[0]!='_']

    # write out the file
    dest = Path(dest)
    dest.mkdir(exist_ok=True)
    hdr = cell_header("00_read.ipynb", Config().path('lib_path'))
    with (dest/'read.py').open('w') as f:
        f.write(create_all_cell(exp_funcs))
        export_cells(cells, hdr, f)

In [ ]:
#hide
path = Path('../nbdev')
path.mkdir(exist_ok=True)
add_init(path)
_export_nb("00_read.ipynb", path)

from nbdev import __version__
from nbdev.read import *
assert __version__,_export_nb

## fin -